Importing dependies

In [42]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

loading the dataset and preview it

In [43]:
movie_df=pd.read_csv('TMDB_new_dataset.csv')

In [44]:
movie_df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,movie_id
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",1
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",2
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",3
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...",4
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",5


In [45]:
movie_df.shape

(12349, 25)

In [46]:
def preprocess_text(text):
    # Lowercase and remove non-alphanumeric characters
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

Selcting the relevent features

In [47]:
selected_features = ['keywords', 'genres', 'title','production_companies','tagline']
for feature in selected_features:
    movie_df[feature] = movie_df[feature].fillna('')
    movie_df[feature] = movie_df[feature].apply(preprocess_text)

Combining the all relevant features

In [48]:
movie_df['combined_features'] = movie_df['keywords'] + ' ' + movie_df['genres'] + ' ' + movie_df['title']+ ' ' + movie_df['production_companies'] + ' ' + movie_df['tagline']

Converting text data to feature vectors

In [49]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(movie_df['combined_features'])

In [50]:
print(feature_vectors)

  (0, 6437)	0.09236552382095989
  (0, 18821)	0.059454281166164934
  (0, 23058)	0.17752491584298805
  (0, 26364)	0.08644090040957948
  (0, 13715)	0.08199072769423997
  (0, 17163)	0.15504525526007193
  (0, 29436)	0.1074424622416639
  (0, 3983)	0.12316796077612109
  (0, 28553)	0.12309452353947012
  (0, 25827)	0.21491630053259536
  (0, 20059)	0.13271547146154924
  (0, 15264)	0.16625506277840285
  (0, 13208)	0.2550284704605907
  (0, 863)	0.08823398697066169
  (0, 9746)	0.09061048585238307
  (0, 23122)	0.09081253698680919
  (0, 744)	0.07820157438744767
  (0, 25406)	0.2550284704605907
  (0, 29159)	0.09921307604888269
  (0, 4311)	0.12842145005177272
  (0, 1448)	0.14257405973771442
  (0, 15761)	0.14097727283077371
  (0, 1799)	0.1970279103805184
  (0, 16848)	0.15751137045199434
  (0, 12185)	0.1652095079628519
  :	:
  (12347, 22431)	0.2012596929334927
  (12347, 28573)	0.3030992802336947
  (12347, 20812)	0.07212679262226555
  (12347, 7972)	0.06386850881620659
  (12348, 12468)	0.19073057373795185
 

Using cosine_similarity algo for finding similarity

In [51]:
similarity = cosine_similarity(feature_vectors)

Exporting the saved and modified dataset

In [52]:
movie_df.to_csv('../../TMDB_model.csv', index=False)

Saving the trained model

In [53]:
# import pickle

# filename='trained_TMDB.sav'
# pickle.dump(similarity,open(filename,'wb'))